# INTELIGENCIA ARTIFICIAL - EVALUACIÓN 1
Integrantes: Sebastian Racedo y Sebastian Valle

Primero importaremos las librerias que necesitarems para llevar a cabo la implementación de nuestro algoritmo.
 - Usaremos la libreria math para operaciones matematicas al momento de calcular la entropia
 - Usaremos la libreria pandas para poder manejar los datasets de una manera optima
 - Usaremos la libreria numpy para las operaciones con matrices

In [1]:
# Libraries Import
import math
import pandas as pd
import numpy as np

Explicar clase Node y como sera el proceso de creación de nodos

In [34]:
class Node:
    def __init__(self, entropy_value, attribute_name, attribute_value, is_final_decision, parent, decision_value="-"):
        self.children = []
        self.entropy_value = entropy_value
        self.decision_value = decision_value
        self.attribute_name = attribute_name
        self.attribute_value = attribute_value
        self.parent = parent
        self.is_final_decision = is_final_decision
        
    def addChild(self, node):
        self.children.append(node)
        
    def getDecisionValues(self):
        if self.is_final_decision:
            return [self.decision_value]
        
        values = set()
        for child in self.children:
            child_values = child.getDecisionValues()
            values = values.union(child_values)
        return values
        
    def areAllChildrenEqual(self):
        if len(self.children) == 0:
            return True
        
        values = self.getDecisionValues()
        return len(values) == 1

La función SplitData recibe como entrada un archivo csv que contiene la informacion del dataset y se encarga de separarla y convertirla en un dataframe lo que la hace mas manejable.
* Input: RawData (Data cruda)
* Output: data convertida en un dataframe, X siendo una matriz que contiene todos los atributos del dataset y Y siendo un vector columna con el atributo a predecir

In [3]:
# Read and split data into training and test sets
def SplitData(Raw_Data):
    data = pd.read_csv(Raw_Data,sep = ',')
    [n,m] = data.shape
    headers = list(data.columns.values)
    X = data.iloc[:,0:m-1]
    y = data.iloc[:,m-1]
    return data, X, y

La función Entropy calcula la entropia de la data ingresada mediante la formula: 
$$ Entropy= -\sum_j p_j*log_2(p_j) $$
Where $p_j$ is the number of ocurrences by events divide by the total number of elements in the dataset

In [4]:
def Entropy(data):
    Size_data = len(data)
    if Size_data == 0:
        return 0
    num_classes = [0 for i in data.iloc[:,-1].dropna().value_counts()]
    num_classes = data.iloc[:,-1].dropna().value_counts().tolist()
    num_classes = [x/Size_data for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num*math.log(num,2)
    return ent*-1

## GainRatio DEBE aceptar valores numericos o NO numericos, Actualmente solo sirve para NO numericos

La función Gain_Ratio recibe como entrada los data a los cuales se le va a calcular la proporción de ganancia y los headers que son los nombres de cada atributo. 

Para calcular las proporciones de ganancia usamos las formulas:

Para la ganancia

$$ ganacia = \text{entropia original G} - \text{ entropia despues de separación por atributo A} $$

La formula de proporción de ganancia es:

$$\text{Gain Ratio} = \frac{ganancia(G,A)}{SplitInfo(G,A)}$$

Donde $SplitInfo(G,A) viene de$

$$SplitInfo(G,A) = - \sum^J_{j=1}\frac{|G_j|}{|G|}log_2\frac{|G_j|}{|G|} $$

Donde $G_j$ es el numero de valores por atributo


Los datos pueden ser tanto nomiales como continuos. 
- Si los datos son nominales:
    - Se divide la data en atributos y por cada atributo se sacan sus valores, a estos valores se les calcula su entropia y se decide cual tiene una mayor proporción de ganancia.
- Si los datos son continuos
    - Algo

La función retorna un vector con la proporción de ganancia por atributo, junto con sus valores

In [5]:
def Gain_Ratio(data,headers):
    List_entropy = []
    List_Sum= []
    List_split = []
    List_values_Per_Attribute = [] 
    Global_Entropy = Entropy(data)
    for attribute in headers[0:len(headers)-1]:
        if (np.issubdtype(data[attribute].dtype, np.number) == False):
            entropy = []
            values = []
            Split = 0
            for value in data[attribute].value_counts().index:
                # Gain Calculation
                values.append(value)
                Neg_branch = data.loc[(data[attribute] == value) & (data[headers[len(headers)-1]] == 'No')]
                Pos_branch  = data.loc[(data[attribute] == value) & (data[headers[len(headers)-1]] == 'Yes')]
                Total_branch = [Neg_branch,Pos_branch]
                Total_branch = pd.concat(Total_branch)
                probability_num = (len(Total_branch)/len(data))
                entropy.append(probability_num*Entropy(Total_branch))
                Split += probability_num*math.log(probability_num,2)
            Split = Split*-1
            List_values_Per_Attribute.append(list(values))
            List_split.append(Split)
            Sum_per_branch = sum(entropy)
            List_entropy.append(list(entropy))
            List_Sum.append(Sum_per_branch)
        else:
            pass
            #print(data.sort_values(by = [attribute]))
    Gain = np.subtract(Global_Entropy,List_Sum)
    GainRatio = np.divide(Gain,List_split)
    return GainRatio, List_values_Per_Attribute

La función SplitAttributes recibe como entrada la data que se dividira junto con los headers de esta, este algoritmo funciona de forma recursiva. Primero se llamara la función Gain_Ratio la cual calculara las proporciones de ganancia, despues se escogere el atributo con la mayor proporción de ganancia y se escogeran sus valores. Despues estos seran pasados a la función GetSubs la cual se encargara de sacaran **los subconjuntos de acuerdo a los valores del atributo que tuvo la mayor proporción de ganancia**, y con estos se volvera a llamar a la función SplitAttributes.

In [41]:
def SplitAttributes(RemainingData, headers, parent, leading_value):
    headers = list(RemainingData.columns.values)
    split_values = []
    if len(headers) == 1:
        yes = len(RemainingData[RemainingData[headers[0]] == "Yes"])
        no = len(RemainingData[RemainingData[headers[0]] == "No"])
        value = None
        if yes > no:
            value = "Yes"
        else:
            value = "No"
        node = Node(decision_value=value, entropy_value=0, attribute_name="Final Decision", attribute_value=leading_value, parent=parent, is_final_decision=True)
        parent.addChild(node)
        return None
    else:
        #print("Current headers " + str(headers))
        GainRatio, values = Gain_Ratio(RemainingData,headers)
        index = np.argmax(GainRatio)
        GainPos = GainRatio[index]
        Best_Attribute = headers[index]
        Best_Values = values[index]
        #print("Best attribute "+ Best_Attribute + " -->")
        #print(GainPos)
        node = Node(entropy_value=GainPos, attribute_name=Best_Attribute, attribute_value=leading_value, parent=parent, is_final_decision=False)
        parent.addChild(node)
        Subs = GetSubs(RemainingData,Best_Attribute,Best_Values)
        #print("Creating node for attribute {} ({}) under parent {}. Number of outcomes: {}".format(Best_Attribute, GainPos, parent.attrib, len(Subs)))
        #print(Best_Values)
        #RemainingData = RemainingData.drop(columns=[Best_Attribute])
        #headers = list(RemainingData.columns.values)
        #print(Subs)
        return [SplitAttributes(Subs[i]["data"], headers, node, Subs[i]["val"]) for i in range(len(Subs))]

La función GetSubs recibe como entrada la data actual, el mejor atributo calculado de la función SplitAttribites y los valores asociados a ese atributo, y devuelve una colección de los dataframes asociados a esos valores.

In [7]:
def GetSubs(data,Best_Attribute, Best_Values):
    data_temp = data.copy()
    dataframe_collection = {} 
    for i in range(len(Best_Values)):
        dataframe_collection[i] = {"val": Best_Values[i], "data": data_temp[data_temp[Best_Attribute] == Best_Values[i]].drop(columns= [Best_Attribute])}
    return dataframe_collection

In [8]:
data, X, y = SplitData("Golf_Other_Way.csv")
data_temp = data.copy()
headers = list(data_temp.columns.values)
print(headers)

['Temp', 'Outlook', 'Humidity', 'Play Golf']


In [39]:
def PrintTree(root, level=0):
    if root.is_final_decision:
        print("{} {}: Decision: {}".format("--" * level, root.attribute_value, root.decision_value))
    else:
        print("{} Node {} ({}) from edge {}. Prunable: {})".format("--" * level, root.attribute_name, root.entropy_value, root.attribute_value, root.areAllChildrenEqual()))
    for child in root.children:
        PrintTree(child, level + 1)

In [28]:
def PruneTree(root):
    if root.areAllChildrenEqual():
        values = list(root.getDecisionValues())
        value = values[0]
        root.children = []
        root.is_final_decision = True
        root.decision_value = value
        root.value = 0
        root.attrib = "Final Decision"
    
    for child in root.children:
        PruneTree(child)

In [43]:
root = Node(attribute_name='-Root-', entropy_value=0, parent=None, attribute_value=None, is_final_decision=False)
Splitter = SplitAttributes(data_temp, headers, root, 0)
Splitter

print("Tree before pruning")
PrintTree(root)

# Prune the tree to remove redundant branches
PruneTree(root)

print()
print("Tree after pruning")
PrintTree(root)

Tree before pruning
 Node -Root- (0) from edge None. Prunable: False)
-- Node Outlook (0.1181062751466107) from edge 0. Prunable: False)
---- Node Humidity (0.04803508424256193) from edge Rainy. Prunable: False)
------ Node Temp (0.31127812445913283) from edge Normal. Prunable: False)
-------- Hot: Decision: No
-------- Cool: Decision: Yes
------ Node Temp (1.0) from edge High. Prunable: False)
-------- Hot: Decision: Yes
-------- Cool: Decision: No
---- Node Temp (-0.0) from edge Overcast. Prunable: True)
------ Node Humidity (-0.0) from edge Hot. Prunable: True)
-------- High: Decision: Yes
-------- Normal: Decision: Yes
------ Node Humidity (-0.0) from edge Cool. Prunable: True)
-------- High: Decision: Yes
-------- Normal: Decision: Yes
---- Node Temp (1.0) from edge Sunny. Prunable: False)
------ Node Humidity (-0.0) from edge Hot. Prunable: True)
-------- High: Decision: No
-------- Normal: Decision: No
------ Node Humidity (-0.0) from edge Cool. Prunable: True)
-------- High: De

### Referencias

[1] https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/lecture-notes/MIT15_097S12_lec08.pdf

[2] https://saiconference.com/Downloads/SpecialIssueNo10/Paper_3A_comparative_study_of_decision_tree_ID3_and_C4.5.pdf

[3] https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5368063